# *実行する前にGPUを選んでください

# ライブラリをインストール

In [1]:
!pip install -U accelerate peft bitsandbytes transformers trl torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.8/514.8 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# ライブラリをロード

In [1]:
import torch
from torch import cuda
from datasets import load_dataset
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
import peft
from peft import LoraConfig
from trl import SFTTrainer

# プロンプトテンプレートの準備

In [2]:
def generate_prompt(data_point):
    if data_point["input"]:
        result = f"[INST] {data_point['instruction']}\n\n{data_point['input']} [/INST] {data_point['output']}"
    else:
        result = f"[INST] {data_point['instruction']} [/INST] {data_point['output']}"
    return result


# text列の追加
def add_text(example):
    example["text"] = generate_prompt(example)
    for key in ["category", "instruction", "input", "output"]:
        del example[key]
    return example

In [3]:
dataset = load_dataset("saldra/sakura_japanese_dataset")
dataset["train"][0]["instruction"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


'5年生は自主的に本の寄付活動に参加しました。 5-1組は500冊、5-2組は5-1組が寄付した本の80％、5-3組は5-2組が寄付した本の120％を寄付しました。 5-1組と5-3組ではどちらが多く本を寄付したでしょうか？ (2通りで比較してください)。'

# データセットの分割

In [4]:
dataset = dataset.map(add_text)
train_val = dataset["train"].train_test_split(
    test_size=0.2, shuffle=True, seed=42
)
train_data = train_val["train"]
val_data = train_val["test"]
train_data[0]["text"]

'[INST] 夫婦用の寝室を持つために都会から引っ越した彼らは、どんな家を求めたか？ 大きな家か郊外の家か大きな家か古い家か私物を置いておくか [/INST] 郊外の家'

# 生成AIをHuggingFaceからロード

In [5]:
hf_auth = 'HF_AUTH_TOKEN'  # ここにHuggingFaceのトークンを入れる
model_id = 'elyza/ELYZA-japanese-Llama-2-7b-instruct'

# 量子化
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4bitのQuantizationの有効化
    bnb_4bit_quant_type="nf4",  # 4bitのQuantizationのタイプ (fp4 or nf4)
    bnb_4bit_compute_dtype=torch.float16,  # 4bitのQuantizationのdtype (float16 or bfloat16)
    bnb_4bit_use_double_quant=False,  # 4bitのDouble-Quantizationの有効化
)

# モデルのロード
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=quantization_config,
    device_map='auto',
    use_auth_token=hf_auth,
    weights_only=True
)
model.config.use_cache = False  # キャッシュ (学習時はFalse)
model.config.pretraining_tp = 1  # 事前学習で使用したTensorランク

Using cuda:0


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/configuration_auto.py:1138: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

## Tokanizerを作成

In [8]:
# Tokenizerの準備
tokenizer = AutoTokenizer.from_pretrained(
    model_id,  # モデル名
    use_fast=False,  # Fastトークナイザーの有効化
    add_eos_token=True,  # データへのEOSの追加を指示
    trust_remote_code=True,
    use_auth_token=hf_auth,
)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = "right" # fp16でのオーバーフロー問題対策

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:902: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

## LoRAの作成

In [6]:
peft_config = LoraConfig(
    r=4,  # LoRAアテンションの次元
    lora_alpha=16,  # LoRAスケーリングのAlphaパラメータ
    lora_dropout=0.05,  # LoRA レイヤーのドロップアウト確率
    bias="none",  # LoRAのバイアス種別 ("none","all", "lora_only")
    task_type="CAUSAL_LM",  # タスク種別
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj", "embed_tokens", "lm_head"],
)
model = peft.get_peft_model(model, peft_config)

# 生成AIをファインチューンする

## トレーナーの作成

In [34]:
eval_steps = 50
save_steps = 400
logging_steps = 400
max_steps = 400 # dollyだと4881
output_dir = './'

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=transformers.TrainingArguments(
        num_train_epochs=3,
        fp16=True,  # fp16学習の有効化
        bf16=False,  # bf16学習の有効化
        optim="paged_adamw_32bit",  # オプティマイザ
        learning_rate=3e-4,   # 初学習率
        lr_scheduler_type="cosine",  # 学習率スケジュール
        max_grad_norm=0.3,  # 最大法線勾配 (勾配クリッピング)
        warmup_ratio=0.03,  # 線形ウォームアップのステップ比率 (0から学習率まで)
        weight_decay=0.001,  # bias/LayerNormウェイトを除く全レイヤーに適用するウェイト減衰
        logging_steps=logging_steps, # nステップ毎にログを記録する
        eval_strategy="steps",
        save_strategy="steps",
        max_steps=max_steps, # 学習ステップ数
        eval_steps=eval_steps, # nステップ毎にEvalをする
        save_steps=save_steps, # nステップ毎にチェックポイントを保存
        output_dir=output_dir, # 出力ディレクトリ
        report_to="none",
        save_total_limit=3,
        push_to_hub=False,
        auto_find_batch_size=True
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


## トレーニング

In [35]:
trainer.train()

Step,Training Loss,Validation Loss
50,No log,nan


Step,Training Loss,Validation Loss
50,No log,nan
100,No log,1.539126
150,No log,1.564688
200,No log,1.312413
250,No log,nan
300,No log,nan
350,No log,nan
400,0.367200,nan


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:220: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


TrainOutput(global_step=400, training_loss=0.3671603012084961, metrics={'train_runtime': 659.8418, 'train_samples_per_second': 1.212, 'train_steps_per_second': 0.606, 'total_flos': 5695314766159872.0, 'train_loss': 0.3671603012084961})

In [ ]:
# LoRAモデルの保存
trainer.model.save_pretrained('myLoRA_model')

# ファインチューンした生成AIを試す

In [36]:
val_data[0]["text"]

'[INST] ジョーは観葉植物を買った。 彼はそれが家のどの部分に似合うと思ったか？ 家族部屋、熱帯林、花畑、映画館、ヘアサロン [/INST] 家族部屋'

観葉植物から植物に変更して試す

In [37]:
prompt = '[INST] ジョーは植物を買った。 彼はそれが家のどの部分に似合うと思ったか？ 家族部屋、熱帯林、花畑、映画館、ヘアサロン [/INST]'

In [38]:
input_ids = tokenizer(prompt, add_special_tokens=False, return_tensors='pt')
output_ids = model.generate(
    **input_ids.to(model.device),
    max_new_tokens=100,
    do_sample=True,
    temperature=0.3,
)
output = tokenizer.decode(output_ids.tolist()[0])
print(output)

[INST] ジョーは植物を買った。 彼はそれが家のどの部分に似合うと思ったか？ 家族部屋、熱帯林、花畑、映画館、ヘアサロン [/INST] 花畑</s>
